In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/g4/t3c38p1x5tx5xd8rs26s4yfm0000gn/T/ipykernel_35062/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [6]:
from sklearn.model_selection import train_test_split

# Assuming the dataset has two columns: 'text' (email content) and 'label' (fraud or not)
X = data.iloc[:, 0]  # Features (email text)
y = data.iloc[:, 1]  # Labels (fraud or not)

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print dataset sizes
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


Training set size: 800
Test set size: 200


## Data Preprocessing

In [9]:
!pip3 install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 17.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [13]:
import re

def clean_html(text):
    # Remove inline JavaScript and CSS
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove all remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove extra spaces, newlines, and tab characters
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply the function to the dataset
data.iloc[:, 0] = data.iloc[:, 0].apply(clean_html)

# Print an example of cleaned text
print(data.iloc[0, 0])


DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL I AM MIKE CHUKWU , THE MANAGER, BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC. I AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT. WE DISCOVERED AN ABANDONED SUM OF $15,000,000.00 (FIFTEEN MILLION UNITED STATES DOLLARS ONLY) IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF A WIFE AND TWO CHILDREN IN NOVEMBER 1997 IN A PLANE CRASH. SINCE WE HEARD OF HIS DEATH, WE HAVE BEEN EXPECTING HIS NEXT-OF-KIN TO COME OVER AND PUT CLAIMS FOR HIS MONEY AS THE HEIR,BECAUSE WE CANNOT RELEASE THE FUND FROM HIS ACCOUNT UNLESS SOMEONE APPLIES FOR CLAIM AS THE NEXT-OF-KIN TO THE DECEASED AS INDICATED IN OUR BANKING GUIDELINES. UNFORTUNATELY, NEITHER THEIR FAMILY MEMBER NOR DISTANT RELATIVE HAS EVER APPEARED TO CLAIM THE SAID FUND. UPON THIS DISCOVERY,I AND OTHER OFFICIALS IN MY DEPARTMENT HAVE 

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [14]:
import re

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove all single characters (like 'a', 'b', etc.)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Remove single characters appearing at the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove prefixed 'b' (from byte conversion artifacts)
    text = re.sub(r'^b\s+', '', text)

    return text

# Apply the cleaning function to the dataset
data.iloc[:, 0] = data.iloc[:, 0].apply(clean_text)

# Print an example of cleaned text
print(data.iloc[0, 0])


dear sir strictly private business proposal am mike chukwu the manager bills and exchange at the foreign remittance department of the zenith international bank plc am writing this letter to ask for your support and cooperation to carry out this business opportunity in my department we discovered an abandoned sum of fifteen million united states dollars only in an account that belongs to one of our foreign customers who died along with his entire family of wife and two children in november in plane crash since we heard of his death we have been expecting his nextofkin to come over and put claims for his money as the heirbecause we cannot release the fund from his account unless someone applies for claim as the nextofkin to the deceased as indicated in our banking guidelines unfortunately neither their family member nor distant relative has ever appeared to claim the said fund upon this discoveryi and other officials in my department have agreed to make business with you and release the 

## Now let's work on removing stopwords
Remove the stopwords.

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# تحميل بيانات stopwords من NLTK
nltk.download('punkt')
nltk.download('stopwords')

# النص الذي ترغب في إزالة الكلمات الشائعة منه
text = "This is a sample sentence with some stopwords like the, and, or."

# تقسيم النص إلى كلمات
words = word_tokenize(text)

# الحصول على قائمة الكلمات الشائعة (stopwords) بالإنجليزية
stop_words = set(stopwords.words('english'))

# إزالة الكلمات الشائعة من النص
filtered_words = [word for word in words if word.lower() not in stop_words and word not in string.punctuation]

# إعادة النص بدون الكلمات الشائعة
filtered_text = " ".join(filtered_words)

print("Original text:", text)
print("Filtered text:", filtered_text)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wafaalqahtani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wafaalqahtani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/wafaalqahtani/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/share/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# تحميل بيانات الـ NLTK المطلوبة
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# النص الذي نريد معالجته
text = "The cats were running across the streets. The dog is barking loudly."

# تقسيم النص إلى كلمات
words = word_tokenize(text)

# تحميل الكلمات الشائعة (stopwords) باللغة الإنجليزية
stop_words = set(stopwords.words('english'))

# تهيئة الـ Lemmatizer
lemmatizer = WordNetLemmatizer()

# إزالة الكلمات الشائعة، وتحويل الكلمات إلى صيغتها الأساسية باستخدام Lemmatization
filtered_lemmatized_words = [
    lemmatizer.lemmatize(word.lower()) 
    for word in words 
    if word.lower() not in stop_words and word not in string.punctuation
]

# إعادة النص بعد تطبيق الـ Lemmatization
filtered_lemmatized_text = " ".join(filtered_lemmatized_words)

print("Original text:", text)
print("Processed text:", filtered_lemmatized_text)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wafaalqahtani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wafaalqahtani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/wafaalqahtani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/wafaalqahtani/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/share/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [30]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

# تحميل البيانات (افترض أن لديك ملف CSV يحتوي على عمود 'message' و 'label')
# البيانات مثال (حاول تحميل البيانات الخاصة بك)
data = pd.read_csv('spam_messages.csv')  # تأكد من المسار الصحيح للملف

# تهيئة أدوات NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# تهيئة الـ Lemmatizer
lemmatizer = WordNetLemmatizer()

# إعداد الكلمات الشائعة
stop_words = set(stopwords.words('english'))

# وظيفة لتنظيف النصوص
def clean_text(text):
    words = word_tokenize(text.lower())  # تحويل النص إلى كلمات صغيرة
    words = [word for word in words if word not in stop_words and word not in string.punctuation]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

# تنظيف البيانات
data['cleaned_message'] = data['message'].apply(clean_text)

# استخدام CountVectorizer لتحويل النصوص إلى تمثيل Bag of Words
vectorizer = CountVectorizer(max_features=10)  # أخذ أعلى 10 كلمات تكرارًا
X = vectorizer.fit_transform(data['cleaned_message'])

# استخراج الكلمات الأكثر تكرارًا
top_words = vectorizer.get_feature_names_out()

# حساب التكرار
word_counts = X.toarray().sum(axis=0)

# عرض النتائج
word_freq = dict(zip(top_words, word_counts))

# عرض أعلى 10 كلمات
print("Top 10 words in ham and spam messages:")
for word, count in word_freq.items():
    print(f"{word}: {count}")

# رسم بياني لأكثر الكلمات تكرارًا
plt.figure(figsize=(10, 6))
plt.bar(word_freq.keys(), word_freq.values(), color='blue')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 10 words in Ham and Spam Messages')
plt.xticks(rotation=45)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'spam_messages.csv'

## Extra features

In [25]:
X_train.head()

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
Name: text, dtype: object

In [24]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

X_train['money_mark'] = X_train['preprocessed_text'].str.contains(money_simbol_list)*1
X_train['suspicious_words'] = X_train['preprocessed_text'].str.contains(suspicious_words)*1
X_train['text_len'] = X_train['preprocessed_text'].apply(lambda x: len(x)) 

X_test['money_mark'] = X_test['preprocessed_text'].str.contains(money_simbol_list)*1
X_test['suspicious_words'] = X_test['preprocessed_text'].str.contains(suspicious_words)*1
X_test['text_len'] = X_test['preprocessed_text'].apply(lambda x: len(x)) 

X_train.head()

KeyError: 'preprocessed_text'

## How would work the Bag of Words with Count Vectorizer concept?

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# بعض النصوص
texts = ["I love Python programming", "Python is great", "I love coding"]

# تهيئة CountVectorizer
vectorizer = CountVectorizer()

# تحويل النصوص إلى مصفوفة Bag of Words
X = vectorizer.fit_transform(texts)

# طباعة معجم الكلمات
print("Vocabulary:", vectorizer.get_feature_names_out())

# طباعة المصفوفة (تمثيل الكلمات بتكرارها)
print("Bag of Words Matrix:")
print(X.toarray())


Vocabulary: ['coding' 'great' 'is' 'love' 'programming' 'python']
Bag of Words Matrix:
[[0 0 0 1 1 1]
 [0 1 1 0 0 1]
 [1 0 0 1 0 0]]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# البيانات (تأكد من تحميل البيانات الخاصة بك)
texts = ["I love Python programming", "Python is great", "I love coding", "Python programming is fun"]

# تهيئة TfidfVectorizer
vectorizer = TfidfVectorizer()

# تحويل النصوص إلى تمثيل TF-IDF
X = vectorizer.fit_transform(texts)

# طباعة الشكل النهائي للمصفوفة (عدد السطور × عدد الأعمدة)
print("Shape of the TF-IDF vectorized dataset:", X.shape)


Shape of the TF-IDF vectorized dataset: (4, 7)


## And the Train a Classifier?

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pandas as pd

# بيانات النصوص والتصنيف (تأكد من استبدالها ببياناتك الحقيقية)
texts = ["I love Python programming", "Python is great", "I love coding", "Python programming is fun"]
labels = ['ham', 'ham', 'spam', 'ham']  # التصنيفات (مثال: ham و spam)

# تهيئة TfidfVectorizer
vectorizer = TfidfVectorizer()

# تحويل النصوص إلى تمثيل TF-IDF
X = vectorizer.fit_transform(texts)

# تقسيم البيانات إلى تدريب واختبار (80% تدريب، 20% اختبار)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# تهيئة نموذج Naive Bayes
model = MultinomialNB()

# تدريب النموذج
model.fit(X_train, y_train)

# التنبؤ بالتصنيفات باستخدام بيانات الاختبار
y_pred = model.predict(X_test)

# تقييم النموذج باستخدام تقارير الأداء
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         ham       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# تحميل البيانات من ملف CSV
# تأكد من تحميل البيانات من Kaggle وتغيير المسار المناسب
data = pd.read_csv("spam_ham_dataset.csv")

# عرض أول 5 صفوف من البيانات للتعرف عليها
print(data.head())

# استخراج النصوص والتصنيفات
texts = data['message']  # افترض أن العمود الذي يحتوي على الرسائل هو 'message'
labels = data['label']   # افترض أن العمود الذي يحتوي على التصنيفات هو 'label'

# تقسيم البيانات إلى تدريب واختبار (80% تدريب، 20% اختبار)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# تهيئة TfidfVectorizer (مع ضبط معلمات إضافية مثل n-grams لتحسين الأداء)
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))  # استخدام bi-grams

# تحويل النصوص إلى تمثيل TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# تهيئة نموذج MultinomialNB (مع المعلمات الافتراضية)
model = MultinomialNB()

# تدريب النموذج
model.fit(X_train_tfidf, y_train)

# التنبؤ بالتصنيفات باستخدام بيانات الاختبار
y_pred = model.predict(X_test_tfidf)

# تقييم النموذج
print(classification_report(y_test, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: 'spam_ham_dataset.csv'